# Chapter 37: Development Tooling

A professional development environment is essential for maintaining code quality, catching errors early, and ensuring consistency across team members. Modern Next.js development requires sophisticated tooling that handles TypeScript, React Server Components, and the unique challenges of full-stack JavaScript applications. Proper configuration of linters, formatters, and debugging tools reduces bugs in production and accelerates development velocity.

By the end of this chapter, you'll master configuring VS Code for optimal Next.js development, implementing ESLint flat configs for modern JavaScript, setting up Prettier with Tailwind class sorting, automating code quality with Git hooks, debugging Server Components and edge runtime, managing environment variables securely, and establishing team-wide development standards.

## 37.1 VS Code Configuration

Optimize Visual Studio Code with extensions and settings tailored for Next.js App Router development.

### Recommended Extensions

```json
// .vscode/extensions.json
{
  "recommendations": [
    // Core Next.js/React support
    "bradlc.vscode-tailwindcss",
    "dbaeumer.vscode-eslint",
    "esbenp.prettier-vscode",
    
    // TypeScript enhancement
    "yoavbls.pretty-ts-errors",
    "usernamehw.errorlens",
    
    // React/Next.js specific
    "antfu.iconify",
    "ms-vscode.vscode-js-profile-flame",
    "Prisma.prisma",
    
    // Testing
    "orta.vscode-jest",
    "ms-playwright.playwright",
    
    // Git and collaboration
    "eamodio.gitlens",
    "github.vscode-pull-request-github",
    
    // Performance and diagnostics
    "wix.vscode-import-cost",
    "pflannery.vscode-versionlens"
  ]
}
```

### Workspace Settings

```json
// .vscode/settings.json
{
  // Editor behavior
  "editor.formatOnSave": true,
  "editor.defaultFormatter": "esbenp.prettier-vscode",
  "editor.codeActionsOnSave": {
    "source.fixAll.eslint": "explicit",
    "source.organizeImports": "explicit"
  },
  
  // TypeScript configuration
  "typescript.tsdk": "node_modules/typescript/lib",
  "typescript.enablePromptUseWorkspaceTsdk": true,
  "typescript.preferences.importModuleSpecifier": "non-relative",
  "typescript.preferences.includePackageJsonAutoImports": "on",
  
  // Tailwind CSS
  "tailwindCSS.includeLanguages": {
    "typescript": "javascript",
    "typescriptreact": "javascript"
  },
  "tailwindCSS.classAttributes": [
    "class",
    "className",
    "classList"
  ],
  "tailwindCSS.experimental.classRegex": [
    ["cva\\(([^)]*)\\)", "[\"'`]([^\"'`]*).*?[\"'`]"],
    ["cx\\(([^)]*)\\)", "(?:'|\"|`)([^']*)(?:'|\"|`)"]
  ],
  
  // File associations
  "files.associations": {
    "*.css": "tailwindcss"
  },
  
  // Search and file exclusion
  "search.exclude": {
    "**/.next": true,
    "**/node_modules": true,
    "**/dist": true,
    "**/.git": true
  },
  "files.exclude": {
    "**/.next": true,
    "**/node_modules": true
  },
  
  // Debugging
  "debug.javascript.autoAttachFilter": "smart",
  "debug.javascript.terminalOptions": {
    "skipFiles": ["<node_internals>/**"]
  }
}
```

### Launch Configuration for Debugging

```json
// .vscode/launch.json
{
  "version": "0.2.0",
  "configurations": [
    {
      "name": "Next.js: Debug Server",
      "type": "node-terminal",
      "request": "launch",
      "command": "npm run dev",
      "cwd": "${workspaceFolder}",
      "internalConsoleOptions": "openOnSessionStart"
    },
    {
      "name": "Next.js: Debug Client",
      "type": "chrome",
      "request": "launch",
      "url": "http://localhost:3000",
      "webRoot": "${workspaceFolder}",
      "sourceMapPathOverrides": {
        "webpack://_N_E/*": "${webRoot}/*",
        "webpack:///./*": "${webRoot}/*"
      }
    },
    {
      "name": "Next.js: Full Stack",
      "type": "node",
      "request": "launch",
      "runtimeExecutable": "npm",
      "runtimeArgs": ["run", "dev"],
      "cwd": "${workspaceFolder}",
      "console": "integratedTerminal",
      "internalConsoleOptions": "neverOpen",
      "serverReadyAction": {
        "pattern": "started server on .+, url: (https?://.+)",
        "uriFormat": "%s",
        "action": "debugWithEdge"
      }
    }
  ]
}
```

## 37.2 ESLint Configuration

Implement the modern flat config format with Next.js, TypeScript, and accessibility rules.

### ESLint Flat Config

```javascript
// eslint.config.mjs
import { dirname } from 'path';
import { fileURLToPath } from 'url';
import { FlatCompat } from '@eslint/eslintrc';
import js from '@eslint/js';
import tsParser from '@typescript-eslint/parser';
import tsPlugin from '@typescript-eslint/eslint-plugin';
import reactPlugin from 'eslint-plugin-react';
import reactHooksPlugin from 'eslint-plugin-react-hooks';
import nextPlugin from '@next/eslint-plugin-next';
import jsxA11yPlugin from 'eslint-plugin-jsx-a11y';
import importPlugin from 'eslint-plugin-import';

const __filename = fileURLToPath(import.meta.url);
const __dirname = dirname(__filename);

const compat = new FlatCompat({
  baseDirectory: __dirname,
});

export default [
  // Base configurations
  js.configs.recommended,
  
  // TypeScript configuration
  {
    files: ['**/*.ts', '**/*.tsx', '**/*.mts', '**/*.cts'],
    languageOptions: {
      parser: tsParser,
      parserOptions: {
        project: true,
        tsconfigRootDir: import.meta.dirname,
      },
    },
    plugins: {
      '@typescript-eslint': tsPlugin,
    },
    rules: {
      ...tsPlugin.configs['strict-type-checked'].rules,
      ...tsPlugin.configs['stylistic-type-checked'].rules,
      '@typescript-eslint/no-misused-promises': [
        'error',
        { checksVoidReturn: { attributes: false } },
      ],
      '@typescript-eslint/no-unused-vars': [
        'error',
        { argsIgnorePattern: '^_', varsIgnorePattern: '^_' },
      ],
      '@typescript-eslint/consistent-type-imports': [
        'error',
        { prefer: 'type-imports', fixStyle: 'inline-type-imports' },
      ],
    },
  },
  
  // React and Next.js configuration
  {
    files: ['**/*.tsx', '**/*.jsx'],
    plugins: {
      react: reactPlugin,
      'react-hooks': reactHooksPlugin,
      '@next/next': nextPlugin,
    },
    languageOptions: {
      parserOptions: {
        ecmaFeatures: { jsx: true },
      },
    },
    settings: {
      react: { version: 'detect' },
    },
    rules: {
      ...reactPlugin.configs.recommended.rules,
      ...reactPlugin.configs['jsx-runtime'].rules,
      ...reactHooksPlugin.configs.recommended.rules,
      ...nextPlugin.configs.recommended.rules,
      ...nextPlugin.configs['core-web-vitals'].rules,
      'react/prop-types': 'off',
      'react/react-in-jsx-scope': 'off',
      'react/no-unknown-property': ['error', { ignore: ['css'] }],
    },
  },
  
  // Accessibility
  {
    files: ['**/*.tsx', '**/*.jsx'],
    plugins: {
      'jsx-a11y': jsxA11yPlugin,
    },
    rules: {
      ...jsxA11yPlugin.configs.recommended.rules,
      'jsx-a11y/alt-text': ['warn', { elements: ['img'], img: ['Image'] }],
      'jsx-a11y/anchor-is-valid': 'off', // Next.js Link handles this
    },
  },
  
  // Import organization
  {
    plugins: {
      import: importPlugin,
    },
    rules: {
      'import/order': [
        'error',
        {
          groups: [
            'builtin',
            'external',
            'internal',
            'parent',
            'sibling',
            'index',
          ],
          'newlines-between': 'always',
          alphabetize: { order: 'asc', caseInsensitive: true },
        },
      ],
      'import/no-duplicates': 'error',
      'import/no-unused-modules': 'warn',
    },
  },
  
  // Specific file patterns
  {
    files: ['**/*.test.ts', '**/*.test.tsx', '**/*.spec.ts', '**/*.spec.tsx'],
    rules: {
      '@typescript-eslint/no-unsafe-call': 'off',
      '@typescript-eslint/no-unsafe-member-access': 'off',
    },
  },
  
  // Ignore patterns
  {
    ignores: [
      '.next/',
      'node_modules/',
      'dist/',
      '*.config.js',
      '*.config.mjs',
      'coverage/',
    ],
  },
];
```

### Legacy ESLint Config (if not using flat config yet)

```javascript
// .eslintrc.json (legacy format - use eslint.config.mjs for new projects)
{
  "extends": [
    "next/core-web-vitals",
    "plugin:@typescript-eslint/strict-type-checked",
    "plugin:@typescript-eslint/stylistic-type-checked"
  ],
  "parser": "@typescript-eslint/parser",
  "parserOptions": {
    "project": true
  },
  "plugins": ["@typescript-eslint"],
  "rules": {
    "@typescript-eslint/consistent-type-imports": [
      "error",
      {
        "prefer": "type-imports",
        "fixStyle": "inline-type-imports"
      }
    ],
    "@typescript-eslint/no-unused-vars": [
      "error",
      { "argsIgnorePattern": "^_", "varsIgnorePattern": "^_" }
    ]
  },
  "overrides": [
    {
      "files": ["*.test.ts", "*.test.tsx"],
      "rules": {
        "@typescript-eslint/no-unsafe-assignment": "off"
      }
    }
  ]
}
```

## 37.3 Prettier and Code Formatting

Configure Prettier for consistent formatting with Tailwind class sorting.

### Prettier Configuration

```javascript
// prettier.config.mjs
/** @type {import("prettier").Config} */
const config = {
  semi: true,
  singleQuote: true,
  tabWidth: 2,
  trailingComma: 'es5',
  printWidth: 80,
  arrowParens: 'always',
  endOfLine: 'lf',
  plugins: [
    'prettier-plugin-tailwindcss',
    '@trivago/prettier-plugin-sort-imports',
  ],
  // Tailwind plugin options
  tailwindAttributes: ['class', 'className', 'classList', 'tw'],
  tailwindFunctions: ['cva', 'cx', 'clsx', 'cn'],
  
  // Import sorting
  importOrder: [
    '^react$',
    '^next(/.*)?$',
    '<THIRD_PARTY_MODULES>',
    '^@/components/(.*)$',
    '^@/lib/(.*)$',
    '^@/hooks/(.*)$',
    '^@/types/(.*)$',
    '^@/(.*)$',
    '^[./]',
  ],
  importOrderSeparation: true,
  importOrderSortSpecifiers: true,
};

export default config;
```

### EditorConfig for Cross-Editor Consistency

```ini
# .editorconfig
root = true

[*]
charset = utf-8
end_of_line = lf
indent_size = 2
indent_style = space
insert_final_newline = true
trim_trailing_whitespace = true

[*.md]
trim_trailing_whitespace = false

[*.{yml,yaml}]
indent_size = 2
```

## 37.4 Git Hooks and Pre-commit Validation

Automate code quality checks before commits to prevent broken code from entering the repository.

### Husky and lint-staged Setup

```bash
# Installation
npm install --save-dev husky lint-staged
npx husky init
```

```javascript
// lint-staged.config.mjs
export default {
  // TypeScript and JavaScript files
  '*.{ts,tsx,js,jsx}': [
    'eslint --fix',
    'prettier --write',
  ],
  
  // JSON, CSS, Markdown
  '*.{json,css,md}': [
    'prettier --write',
  ],
  
  // Type checking for staged files only
  '*.{ts,tsx}': () => 'tsc --noEmit --skipLibCheck',
};
```

```bash
# .husky/pre-commit
#!/bin/sh
. "$(dirname "$0")/_/husky.sh"

npx lint-staged
```

```bash
# .husky/commit-msg
#!/bin/sh
. "$(dirname "$0")/_/husky.sh"

# Commit message validation (Conventional Commits)
npx --no -- commitlint --edit ${1}
```

### Commitlint Configuration

```javascript
// commitlint.config.js
module.exports = {
  extends: ['@commitlint/config-conventional'],
  rules: {
    'type-enum': [
      2,
      'always',
      [
        'feat',     // New feature
        'fix',      // Bug fix
        'docs',     // Documentation
        'style',    // Formatting (no code change)
        'refactor', // Code refactoring
        'perf',     // Performance improvement
        'test',     // Adding tests
        'chore',    // Maintenance
        'ci',       // CI/CD changes
        'build',    // Build system
        'revert',   // Revert commit
      ],
    ],
    'subject-case': [2, 'never', ['start-case', 'pascal-case']],
    'subject-full-stop': [2, 'never', '.'],
    'header-max-length': [2, 'always', 72],
  },
};
```

## 37.5 TypeScript Strict Configuration

Enforce maximum type safety with strict compiler options.

```json
// tsconfig.json
{
  "compilerOptions": {
    // Target and Module
    "target": "ES2022",
    "lib": ["dom", "dom.iterable", "ES2022"],
    "allowJs": true,
    "skipLibCheck": true,
    "strict": true,
    "noEmit": true,
    "esModuleInterop": true,
    "module": "ESNext",
    "moduleResolution": "bundler",
    "resolveJsonModule": true,
    "isolatedModules": true,
    "jsx": "preserve",
    "incremental": true,
    "plugins": [
      {
        "name": "next"
      }
    ],
    
    // Strictness options
    "strictNullChecks": true,
    "strictFunctionTypes": true,
    "strictBindCallApply": true,
    "strictPropertyInitialization": true,
    "noImplicitThis": true,
    "alwaysStrict": true,
    "noUnusedLocals": true,
    "noUnusedParameters": true,
    "noImplicitReturns": true,
    "noFallthroughCasesInSwitch": true,
    "noUncheckedIndexedAccess": true,
    "exactOptionalPropertyTypes": true,
    
    // Path mapping
    "baseUrl": ".",
    "paths": {
      "@/*": ["./*"],
      "@/components/*": ["./components/*"],
      "@/lib/*": ["./lib/*"],
      "@/hooks/*": ["./hooks/*"],
      "@/app/*": ["./app/*"],
      "@/types/*": ["./types/*"]
    },
    
    // Next.js specific
    "forceConsistentCasingInFileNames": true,
    "verbatimModuleSyntax": true
  },
  "include": ["next-env.d.ts", "**/*.ts", "**/*.tsx", ".next/types/**/*.ts"],
  "exclude": ["node_modules"]
}
```

## 37.6 Debugging Server Components

Configure debugging for React Server Components and the App Router.

### VS Code Debug Configuration

```json
// .vscode/launch.json (Server Component debugging)
{
  "version": "0.2.0",
  "configurations": [
    {
      "name": "Debug Server Components",
      "type": "node",
      "request": "launch",
      "runtimeExecutable": "node",
      "runtimeArgs": [
        "--inspect",
        "./node_modules/next/dist/bin/next",
        "dev"
      ],
      "env": {
        "NODE_OPTIONS": "--inspect"
      },
      "port": 9229,
      "sourceMaps": true,
      "sourceMapPathOverrides": {
        "webpack:///./*": "${workspaceFolder}/*",
        "webpack:///src/*": "${workspaceFolder}/src/*",
        "webpack:///*": "*"
      }
    },
    {
      "name": "Debug Edge Runtime",
      "type": "node",
      "request": "attach",
      "port": 9230,
      "restart": true,
      "sourceMaps": true
    }
  ]
}
```

### Console Logging for Server Components

```typescript
// lib/debug/server-logger.ts
import { headers } from 'next/headers';

export function serverLog(message: string, data?: any) {
  if (process.env.NODE_ENV === 'development') {
    const timestamp = new Date().toISOString();
    const requestId = crypto.randomUUID().slice(0, 8);
    
    console.log(
      `[Server:${requestId}] ${timestamp} - ${message}`,
      data ? JSON.stringify(data, null, 2) : ''
    );
  }
}

// Usage in Server Components
import { serverLog } from '@/lib/debug/server-logger';

export default async function DashboardPage() {
  serverLog('Rendering dashboard', { userId: '123' });
  
  const data = await fetchData();
  serverLog('Data fetched', { count: data.length });
  
  return <Dashboard data={data} />;
}
```

### React DevTools Integration

```typescript
// lib/debug/react-devtools.tsx (Client Component)
'use client';

import { useEffect } from 'react';

export function ReactDevTools() {
  useEffect(() => {
    if (process.env.NODE_ENV === 'development') {
      // Ensure React DevTools can connect
      if (typeof window !== 'undefined') {
        // @ts-ignore
        window.__REACT_DEVTOOLS_GLOBAL_HOOK__ = window.__REACT_DEVTOOLS_GLOBAL_HOOK__ || {};
      }
    }
  }, []);

  return null;
}

// app/layout.tsx
import { ReactDevTools } from '@/lib/debug/react-devtools';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en">
      <body>
        {children}
        {process.env.NODE_ENV === 'development' && <ReactDevTools />}
      </body>
    </html>
  );
}
```

## 37.7 Environment Management

Securely manage environment variables across different deployment targets.

### Environment Validation

```typescript
// lib/env.ts
import { z } from 'zod';

const envSchema = z.object({
  // Server-side variables
  DATABASE_URL: z.string().url(),
  NEXTAUTH_SECRET: z.string().min(32),
  OPENAI_API_KEY: z.string().startsWith('sk-').optional(),
  
  // Client-side variables (must be prefixed with NEXT_PUBLIC_)
  NEXT_PUBLIC_APP_URL: z.string().url(),
  NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY: z.string().startsWith('pk_'),
  
  // Feature flags
  ENABLE_ANALYTICS: z.enum(['true', 'false']).default('false'),
  NODE_ENV: z.enum(['development', 'production', 'test']),
});

// Validate on import (fails fast)
const parsed = envSchema.safeParse(process.env);

if (!parsed.success) {
  console.error('❌ Invalid environment variables:', parsed.error.flatten().fieldErrors);
  throw new Error('Invalid environment variables');
}

export const env = parsed.data;

// Type-safe access
declare global {
  namespace NodeJS {
    interface ProcessEnv extends z.infer<typeof envSchema> {}
  }
}
```

### Environment Types Declaration

```typescript
// types/env.d.ts
declare namespace NodeJS {
  interface ProcessEnv {
    // Database
    DATABASE_URL: string;
    DIRECT_URL: string;
    
    // Authentication
    NEXTAUTH_SECRET: string;
    NEXTAUTH_URL: string;
    GOOGLE_CLIENT_ID: string;
    GOOGLE_CLIENT_SECRET: string;
    
    // Payments
    STRIPE_SECRET_KEY: string;
    STRIPE_WEBHOOK_SECRET: string;
    NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY: string;
    
    // Storage
    S3_BUCKET_NAME: string;
    AWS_REGION: string;
    
    // AI
    OPENAI_API_KEY: string;
    
    // App Config
    NEXT_PUBLIC_APP_URL: string;
    NODE_ENV: 'development' | 'production' | 'test';
  }
}
```

## Key Takeaways from Chapter 37

1. **VS Code Configuration**: Install extensions for Tailwind CSS, Pretty TypeScript Errors, and ESLint. Configure `typescript.tsdk` to use the workspace TypeScript version, and set up debugging launch configurations for both Server Components (Node.js debugger on port 9229) and Client Components (Chrome debugger).

2. **ESLint Flat Config**: Migrate to `eslint.config.mjs` (ESLint v9+) for modern configuration. Include `@typescript-eslint` for strict type checking, `@next/next` for framework rules, and `jsx-a11y` for accessibility. Use the `FlatCompat` utility to bridge legacy plugins during migration.

3. **Prettier Integration**: Use `prettier-plugin-tailwindcss` to automatically sort Tailwind classes in the recommended order (layout → sizing → spacing → colors). Configure `@trivago/prettier-plugin-sort-imports` to organize imports by groups (React/Next → third-party → internal → relative) with newline separators.

4. **Git Hooks**: Implement Husky with `lint-staged` to run ESLint and Prettier only on staged files, ensuring fast pre-commit checks. Add `commitlint` to enforce Conventional Commits (feat:, fix:, docs:, etc.) for automated changelog generation.

5. **TypeScript Strictness**: Enable `strict: true`, `noUncheckedIndexedAccess`, and `exactOptionalPropertyTypes` in `tsconfig.json` to catch null pointer exceptions and undefined property access at compile time. Use path mapping (`@/*`) for clean imports.

6. **Debugging**: Attach to the Node.js inspector (port 9229) for debugging Server Components and API routes. Use `serverLog` utility functions that only execute in development to trace data fetching without affecting production performance.

7. **Environment Validation**: Use Zod to validate `process.env` at startup, failing fast with descriptive error messages if required variables are missing or malformed. Separate client variables (`NEXT_PUBLIC_*`) from server secrets and never log sensitive values.

## Coming Up Next

**Chapter 38: Turbopack Deep Dive**

With your development environment optimized for productivity and code quality, it's time to explore Next.js's new build engine. In Chapter 38, we'll dive deep into Turbopack's incremental compilation architecture, migration strategies from Webpack, advanced configuration options, caching mechanisms, and troubleshooting techniques. You'll learn how to leverage this Rust-powered bundler for faster development iterations and optimized production builds.